# Programming Fortran With GNU Fortran

## 1. gfortran - the GNU Fortran compiler

https://gcc.gnu.org/wiki/GFortran

Gfortran is the name of the GNU Fortran project, developing a free Fortran 95/2003/2008 compiler for GCC, the GNU Compiler Collection. The gfortran development effort uses an open development environment in order to attract a larger team of developers and to ensure that gfortran can work on multiple architectures and diverse environments. 



#### Debian/Ubuntu

In [ ]:
!sudo apt install gfortran

#### Windows

you have gfortran after install **MinGW-GW64**

you may verify the Gfortran installation by listing the version of Gfortran: 

In [ ]:
!gfortran --version

## 2  Getting Started

Compile/Link a Simple Fortran Program - hello.f90

> **NOTE**: .f90 for **free form source Fortran**

In [ ]:
%%file ./code/gfortran/hello.f90

program hello
  write(*,*) 'Fortran says Hello, world!'
end program hello

Then,use **gfortran** to the compile Fortran program

In [ ]:
!gfortran -c ./code/gfortran/hello.f90
!gfortran -o ./code/gfortran/hello hello.o

In [ ]:
!dir hell*.o
!dir ./code/gfortran/hello.*

In [ ]:
!./code/gfortran/hello

## 3 Procedures

`Subroutines` and `functions` are called procedures

### 3.1 Subroutines

* use the `call` statement, `call <sub name>(args...)`

The subroutine is defined somewhere else:

```
subroutine <name>(args)
use ...
implicit none
variable declarations
executable statements
end subroutine <name>
```
**Dummy Arguments**

* Variable names in a procedure, only exist in that procedure

* Fortran is `pass-by-reference`

### 3.2 Functions

Purpose is to return **only one** result

The **function name** 

* defines the variable in the function

* is returned at the end of a function


### 3.3 Example Procedures
```
 ./code/gfortran/bisect/
```

In [ ]:
%%file ./code/gfortran/bisect/bisect.f08
subroutine bisect(fun,xl,xr,x,f,tol)

implicit none

  ! arguments
  real(8),intent(inout) :: xl       ! left bound
  real(8),intent(inout) :: xr       ! right bound
  real(8),intent(inout) :: x        ! result
  real(8),intent(out)   :: f        ! residual
  real(8),intent(in)    :: tol      ! residual tolerance

  ! function argument
  real(8) :: fun
  external fun

  ! local varaibles
  real(8) :: fl     ! residual for left  bound
  real(8) :: fr     ! resdiual for right bound
  integer :: i      ! loop counter

  ! determine residual bounds
  fl = fun(xl)
  fr = fun(xr)

  ! begin loop
  do i = 1,100

    ! get midpoint
    x = 0.5_8*(xl + xr)

    ! evaluate resdiual at midpoint
    f = fun(x)

    ! check for convergence
    if (abs(f) < tol) exit

    ! reset the bounds
    if (f*fl < dble(0.0)) then

      ! move right bound info to mid
      xr = x
      fr = f

    else

      ! move left bound info to mid
      xl = x
      fl = f

    end if


    ! print out information
    print *,'Iteration:',i,' Residual:',f

  end do

end subroutine bisect


### 3.4 Example Function

In [ ]:
%%file ./code/gfortran/bisect/myfun.f08
function myfun(x)

  implicit none

  ! formal variables
  real(8) :: myfun   ! the function declaration, residual
  real(8) :: x       ! the independent variable

  myfun = x**2 - 4

end function myfun

### 3.5 Main demo

In [ ]:
%%file ./code/gfortran/bisect/nonlinear.f08
program nonlinear

  implicit none

  real(8) :: x   ! solution variable
  real(8) :: xl  ! lower bound
  real(8) :: xr  ! upper bound
  real(8) :: f   ! final residual
  real(8) :: tol ! tolerance

  ! function declaration
  external myfun

  ! ask user for guess
  x=1.2

  ! ask for bounds
  xl=0.1
  xr=2.0

  ! ask for tolerance
  tol=0.01

  ! solve by bisection
  call bisect(myfun,xl,xr,x,f,tol)

  ! print result
  print *,'Solution is:',x,' with final residual:',f

  ! terminate the program
  stop

end program nonlinear


In [ ]:
%%file ./code/gfortran/bisect/makefile

DIR= ./code/gfortran/bisect/
program = nonlinear 
F08 = gfortran

$(program):
	$(F08) -o $(DIR)bisect.o -c $(DIR)bisect.f08  
	$(F08) -o $(DIR)myfun.o -c $(DIR)myfun.f08 
	$(F08) -o $(DIR)nonlinear.o -c $(DIR)nonlinear.f08 
	$(F08) -o $(DIR)nonlinear.exe $(DIR)nonlinear.o  $(DIR)bisect.o  $(DIR)myfun.o  
	del .\code\gfortran\bisect\*.o

In [ ]:
!make -f ./code/gfortran/bisect/makefile

In [ ]:
!.\code\gfortran\bisect\nonlinear.exe 

## 4 Modules

Modules fulfill multiple purposes

Used for shared declarations (similar to headers)

Used for defining global data

Used for defining procedure interfaces

Think of them as a high-level interface in your code

Structure of a Module

```fortran
module <name>
use definitions
implicit none
static data definitions, global to the module
contains
procedure definitions and interfaces
end module <name>
```
**Example module class_Circle**

```
./code/gfortran/circle/
```

In [ ]:
%%file ./code/gfortran/circle/class_Circle.f08
module class_Circle

  implicit none
  private
  public :: circle_print

  real :: pi = 3.1415926535897931D0 ! module-wide private constant

  type, public :: Circle

    real :: radius ! the radius of a circle
    real :: area   ! the area of a circle
    real :: circum ! the circumference of a circle

  end type Circle

contains

!==============================================================================
! CIRCLE_AREA calculates the area of a circle
!==============================================================================

  function circle_area(this) result(area)

    type(Circle), intent(in)  :: this ! circle instance
    real                      :: area ! the area of the circle

    ! calculate the area
    area = pi * this%radius**2

  end function circle_area

!==============================================================================
! CIRCLE_CIRCUM calculate the circumference of a circle
!==============================================================================

  function circle_circum(this) result(circum)

    type(Circle), intent(in) :: this   ! circle instance
    real                     :: circum ! the circumference

    ! calculate circumference
    circum = 2*pi*this%radius

  end function circle_circum

!==============================================================================
! CIRCLE_PRINT prints information about the circle
!==============================================================================

  subroutine circle_print(this)

    type(Circle), intent(inout) :: this ! circle instance

    ! calculate area
    this%area = circle_area(this)

    ! calculate circumference
    this%circum = circle_circum(this)

    ! print results
    write(*,'("The area is:",T30,F0.4,/,"The circumference is:",T30,F0.4)')    &
   &           this%area,this%circum

  end subroutine circle_print

end module class_Circle


In [ ]:
%%file ./code/gfortran/circle/circle_test.f08
program circle_test

  use class_Circle, only: Circle,circle_print

  implicit none

  type(Circle) :: acircle  ! a circle instance

  acircle%radius=3.2
 
  write(*,'("The radius is:",T30,F0.4)') acircle%radius
  ! print out results
  call circle_print(acircle)

  ! terminate the program
  stop

end program circle_test


In [ ]:
%%file ./code/gfortran/circle/makefile

DIR= ./code/gfortran/circle/

all: circle_test

circle_test: obj
	 gfortran -o $(DIR)circle_test.exe  $(DIR)circle_test.o $(DIR)class_Circle.mod
	 del .\code\gfortran\circle\*.o
	 del .\code\gfortran\circle\*.mod    
    
obj:
	 gfortran -o $(DIR)class_Circle.mod -c  $(DIR)class_Circle.f08    
	 gfortran -o $(DIR)circle_test.o  -c  $(DIR)circle_test.f08     


In [ ]:
!make -f ./code/gfortran/circle/makefile

In [ ]:
!.\code\gfortran\circle\circle_test

## Reference

1. MIT 22.901 Fortran Class https://github.com/bhermanmit/Fortran

2. Awesome list of Fortran libs https://github.com/rabbiabram/awesome-fortran

3. SEUIF97-API Fortran： https://github.com/PySEE/SEUIF97/tree/master/demo-fortran